# LangGraph Integration

This notebook demonstrates building agentic workflows with LangGraph: nodes, edges, state management, and conditional routing.

## Setup

In [ ]:
!pip install langchain langchain-openai langgraph python-dotenv -q

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in environment variables. Please create a .env file with your API key.")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

## Section 1: LangGraph Basics

In [ ]:
# Define state
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

# Define nodes
def process_input(state: AgentState):
    return {"messages": [("user", "Processed input")]}

def generate_response(state: AgentState):
    response = llm.invoke("Generate a response")
    return {"messages": [("assistant", response.content)]}

# Create graph
workflow = StateGraph(AgentState)
workflow.add_node("process", process_input)
workflow.add_node("respond", generate_response)
workflow.set_entry_point("process")
workflow.add_edge("process", "respond")
workflow.add_edge("respond", END)

# Compile and run
app = workflow.compile()
result = app.invoke({"messages": []})
print("Graph executed successfully")

**Scoping Insight**: Experience the complexity - recognize when a simple graph is enough vs when you need more. LangGraph adds structure but also overhead. Use it when you need multi-step workflows, not for simple single-step tasks.

## Section 2: State Management

In [ ]:
# State with multiple fields
class ComplexState(TypedDict):
    query: str
    results: list
    step_count: int

def search_node(state: ComplexState):
    # Update state
    return {
        "query": state["query"],
        "results": ["result1", "result2"],
        "step_count": state.get("step_count", 0) + 1
    }

def analyze_node(state: ComplexState):
    # Use state from previous node
    print(f"Analyzing {len(state['results'])} results for query: {state['query']}")
    return {"step_count": state["step_count"] + 1}

workflow2 = StateGraph(ComplexState)
workflow2.add_node("search", search_node)
workflow2.add_node("analyze", analyze_node)
workflow2.set_entry_point("search")
workflow2.add_edge("search", "analyze")
workflow2.add_edge("analyze", END)

app2 = workflow2.compile()
result = app2.invoke({"query": "test query", "results": [], "step_count": 0})
print(f"Final step count: {result['step_count']}")

**Scoping Insight**: Understand state management overhead - when it's necessary vs when it adds unnecessary complexity. State management is powerful but requires careful design. Use it when you need to track information across steps, not for stateless operations.

## Section 3: Conditional Edges

In [ ]:
# Conditional routing based on state
def should_continue(state: AgentState):
    messages = state.get("messages", [])
    if len(messages) > 5:
        return "end"
    return "continue"

def continue_node(state: AgentState):
    return {"messages": [("assistant", "Continuing...")]}

workflow3 = StateGraph(AgentState)
workflow3.add_node("continue", continue_node)
workflow3.set_entry_point("continue")
workflow3.add_conditional_edges(
    "continue",
    should_continue,
    {
        "continue": "continue",  # Loop back
        "end": END
    }
)

app3 = workflow3.compile()
print("Conditional routing workflow created")

**Scoping Insight**: Recognize when conditional logic is essential vs when linear flows work. Conditional edges add complexity but enable dynamic workflows. Use them when you need branching logic, not for simple sequential processes.

## Section 4: Agentic Patterns

In [ ]:
# Agent with tool calling in nodes
from langchain.tools import Tool

def tool_node(state: AgentState):
    # Simulate tool execution
    tool_result = "Tool executed successfully"
    return {"messages": [("tool", tool_result)]}

def agent_node(state: AgentState):
    # Agent decides next action
    response = llm.invoke("What should I do next?")
    return {"messages": [("assistant", response.content)]}

workflow4 = StateGraph(AgentState)
workflow4.add_node("agent", agent_node)
workflow4.add_node("tool", tool_node)
workflow4.set_entry_point("agent")
workflow4.add_edge("agent", "tool")
workflow4.add_edge("tool", END)

app4 = workflow4.compile()
print("Agentic workflow with tool calling created")

**Scoping Insight**: Hands-on experience with agent limitations - when agents work well vs when they fail. Agents are powerful but unpredictable. Test thoroughly and have fallback plans. Use agents when you need autonomous decision-making, not for deterministic processes.

## Section 5: Advanced Examples

In [ ]:
# Multi-agent system pattern
class MultiAgentState(TypedDict):
    task: str
    supervisor_decision: str
    worker_result: str

def supervisor_node(state: MultiAgentState):
    # Supervisor decides which worker to use
    decision = "worker_a" if "analyze" in state["task"] else "worker_b"
    return {"supervisor_decision": decision}

def worker_a_node(state: MultiAgentState):
    return {"worker_result": "Analysis complete"}

def worker_b_node(state: MultiAgentState):
    return {"worker_result": "Processing complete"}

def route_worker(state: MultiAgentState):
    return state["supervisor_decision"]

workflow5 = StateGraph(MultiAgentState)
workflow5.add_node("supervisor", supervisor_node)
workflow5.add_node("worker_a", worker_a_node)
workflow5.add_node("worker_b", worker_b_node)
workflow5.set_entry_point("supervisor")
workflow5.add_conditional_edges("supervisor", route_worker, {"worker_a": "worker_a", "worker_b": "worker_b"})
workflow5.add_edge("worker_a", END)
workflow5.add_edge("worker_b", END)

app5 = workflow5.compile()
print("Multi-agent system created")

**Scoping Insight**: Experience the complexity of advanced patterns - recognize when to recommend simpler solutions or subcontract. Multi-agent systems are powerful but complex. Use them when you need parallel processing or specialized agents, not as a default. Consider subcontracting for very complex implementations.